## STEPS TO BE FOLLOWED 

- Notebook to Create Connection Between Databricks to MYSQL and BigQuery
- Reads the conig_table rows for **active_flag=1 AND load_flag=1**
- Export the Each **BigQuery Table to --> GCS Stagging Layer**
- Update and Log the status of the Load in config_table

In [0]:
dbutils.widgets.text("mysql_host","136.113.73.13")
mysql_host=dbutils.widgets.get("mysql_host")

dbutils.widgets.text("mysql_port","3306")
mysql_port=dbutils.widgets.get("mysql_port")

dbutils.widgets.text("mysql_user","root")
mysql_user=dbutils.widgets.get("mysql_user")

dbutils.widgets.text("mysql_password","Admin@1234")
mysql_password=dbutils.widgets.get("mysql_password")

dbutils.widgets.text("bq_sa_key","/Workspace/Users/anurag.srivastava@koantekorg.onmicrosoft.com/Migrationproject/datamigrationproject-475415-ffc1c7e9a773.json")
bq_sa_key=dbutils.widgets.get("bq_sa_key")

In [0]:
%pip install mysql-connector-python google-cloud-bigquery


In [0]:
import json,os,datetime as dt
import mysql.connector as mc
from google.cloud import bigquery
from contextlib import contextmanager

#Enclosing it in Dictiornary
MYSQL={
    'host':mysql_host.strip(),
    'port':int(mysql_port),
    'user':mysql_user.strip(),
    'pwd':mysql_password
}

#Repalcing the dbfs:/ with /dbfs/`
BQ_KEY=bq_sa_key.replace('dbfs:/','/dbfs/')